In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
data = pd.read_excel('Region_NW.xlsx', index_col=1)
data.index = pd.to_datetime(data.index)
data = data[['D']]
data = data.dropna()
temp = data['D'].pct_change()
temp.columns = ['delta_D']
temp.rename("delta_D", inplace=True)
data = data.join(temp, how='right')
temp = temp.pct_change()
temp.columns = ['delta_2_D']
temp.rename("delta_2_D", inplace=True)
data = data.join(temp, how='right')

In [3]:
regions = ['BPAT', 'NEVP', 'PACE', 'PACW', 'PSCO', 'WACM', 'PSEI']
for r in regions:
    region = pd.read_excel(r + '.xlsx', index_col=1)
    region.index = pd.to_datetime(region.index)
    idx = (region['Imputed D'].notna()) & (region['D'].isna())
    region.loc[idx, 'D'] = region.loc[idx, 'Imputed D']
    idx = (region['Imputed D'].isna()) & (region['D'].isna()) & (region['DF'].notna())
    region.loc[idx, 'D'] = region.loc[idx, 'DF']
    temp = region[['D']]
    data = data.join(temp, rsuffix='_'+r)
    # Prepare the data
    complete_data = data[data['D_'+r].notna()]
    missing_data = data[data['D_'+r].isna()]

    # Perform linear regression
    X_complete = complete_data[['D']]
    y_complete = complete_data['D_'+r]
    model = LinearRegression()
    model.fit(X_complete, y_complete)

    # Impute the missing values
    X_missing = missing_data[['D']]
    if len(X_missing) == 0:
        continue
    predicted = model.predict(X_missing)

    # Replace the missing values in the original dataset
    data.loc[data['D_'+r].isna(), 'D_'+r] = predicted

In [4]:
data = data.dropna()
data['delta_D_0'] = np.where(data['delta_D'] == 0, 1, 0)
data = data.replace([np.inf, -np.inf], np.nan)  # replace inf with NaN
data = data.fillna(method='ffill', axis=0)  # fill NaN with previous value
cols_to_standardize = data.columns[:-1]
scaler = StandardScaler()
scaler.fit(data[cols_to_standardize])
data[cols_to_standardize] = scaler.transform(data[cols_to_standardize])
data.to_csv('data.csv')